In [15]:
import great_expectations as gx
import pandas as pd
import warnings
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime, date
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# Load the data
#df = pd.read_csv("./data/transactions.csv")
transaction_df = pd.read_csv("./data/transactions.csv", dtype="string")
account_df = pd.read_csv("./data/sebank_customers_with_accounts.csv", dtype="string")

for column in transaction_df:
    if column == "amount": #removes spacing errors from the amount column
        transaction_df[column] = transaction_df[column].astype(str).str.replace(" ","")
    elif column == "timestamp":
        transaction_df[column] = transaction_df[column].astype(str).str.replace("-","")
        transaction_df[column] = transaction_df[column].astype(str).str.replace(".",":")
    transaction_df[column] = transaction_df[column].astype(str).str.strip()
    print(transaction_df[column].head())

#data conversion
transaction_df["amount"] = pd.to_numeric(transaction_df["amount"], errors="coerce")
for index in range(transaction_df.shape[0]):
    temp_list = str(transaction_df.loc[index,'timestamp']).split()
    temp_date = temp_list[0]
    temp_time = temp_list[1]
    datetime_corrected = False
    if len(temp_list[0]) == 6:
        datetime_corrected = True
        temp_date = "20" + temp_list[0]
        if datetime.strptime(temp_date, "%Y%m%d").date() > date.today():
            temp_date = "19" + temp_list[0]
            #end of date length check if-statement
    if len(temp_list[1]) == 5:
        datetime_corrected = True
        temp_time = temp_list[1] + ":00"
        #end of date length check if-statement
    if datetime_corrected:
        temp_timestamp = temp_date + " " + temp_time
        transaction_df.loc[index,'timestamp'] = temp_timestamp

    transaction_df.loc[index,'timestamp'] = pd.to_datetime(transaction_df.loc[index,'timestamp'], format="%Y%m%d %H:%M:%S")

    #if str(transaction_df.loc[index,'timestamp']).startswith('20'):
        #transaction_df.loc[index,'timestamp'] = pd.to_datetime(transaction_df.loc[index,'timestamp'], format="%Y%m%d %H:%M:%S")
    #else:
        #print(f"{str(transaction_df.loc[index,'timestamp'])[0:2]} {transaction_df.loc[index,'timestamp']}")
        #transaction_df.loc[index,'timestamp'] = pd.to_datetime(transaction_df.loc[index,'timestamp'], format="%y%m%d %H:%M:%S")

# Create the ephemeral GX context
context = gx.get_context()

# Add a pandas datasource
data_source = context.data_sources.add_pandas(name="pandas")

# Add a dataframe asset
data_asset = data_source.add_dataframe_asset(name="transactions_data")

# Define the batch (entire DataFrame)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

# Create the expectation suite with a name
suite = gx.core.expectation_suite.ExpectationSuite(name="transactions_suite")

# Get the validator using the suite
validator = context.get_validator(batch=batch, expectation_suite=suite)

# Add expectations
validator.expect_column_values_to_be_between("amount", min_value=0.01, max_value=100000)
validator.expect_column_values_to_not_be_null("timestamp")

# Validate
results = validator.validate()

# Print results
print(results)

0    f2f3b0fc-b7d7-4d85-b682-cbf07ef77c1a
1    feccbd1e-7752-4452-a44a-901510252561
2    bc6a480c-e2e1-434d-830c-1cda3f913038
3    b832aa2c-77f3-4c4f-9697-9776b8d74abb
4    0d32c6ef-508e-4f5d-bc64-341a0a9f498d
Name: transaction_id, dtype: object
0    20250306 12:04:00
1    20250331 01:37:00
2    20250429 22:58:00
3    20250227 14:52:00
4    20250103 22:29:00
Name: timestamp, dtype: object
0     7746.03
1    45193.04
2    33029.71
3    35994.61
4    32229.73
Name: amount, dtype: object
0    SEK
1    SEK
2    SEK
3    SEK
4    SEK
Name: currency, dtype: object
0    SE8902EWFT80524424320740
1    SE8902GTSI58472323675409
2    SE8902SEZT63377163386089
3    SE8902MGEE43594824630478
4    SE8902SZWO66860675966705
Name: sender_account, dtype: object
0    SE8902ZUIH32054423564254
1    SE8902ERNA19462716907689
2    SE8902LOLQ30361773270969
3    SE8902XIVG18316797897983
4    SE8902SUUS97732018069804
Name: receiver_account, dtype: object
0    Sweden
1    Sweden
2    Sweden
3    Sweden
4    Sweden
N

ValueError: time data "20250131 24:30:00" doesn't match format "%Y%m%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.